# Imports

In [1]:
import os, time

import numpy as np
import pandas as pd

from matplotlib import pyplot as plt
import seaborn as sns
from plotly import express as px

# import nltk
from nltk.tokenize import word_tokenize

from sklearn.model_selection import train_test_split

import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn, optim

torch.__version__

'2.5.1'

setups

In [2]:
# nltk.download('punkt')
# nltk.download('punkt_tab')

In [3]:
torch.manual_seed(44)

if torch.cuda.is_available():
    device = torch.device("cuda")
    print("CUDA is available, so using", torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("CUDA is not available, so using CPU only.")

CUDA is available, so using NVIDIA GeForce GTX 1660 Ti


# Data Loading

In [4]:
with open("Data/Other/campusx-faq-dataset.txt", "r", encoding="utf-8") as f:
    document = f.read()

document = document.lower()

# Data Preparation

tokenize

In [5]:
document = document.lower()

tokens = word_tokenize(document)
unique_tokens = list(set(tokens))

len(tokens), len(unique_tokens)

(1018, 288)

vocab space

In [6]:
vocab_space = {'<UNK>': 0} # for out of vocab

for token in unique_tokens:
    if not vocab_space.get(token):
        vocab_space[token] = len(vocab_space)

len(vocab_space)

289

vectors

In [7]:
def text_to_vector(text, vocab_space):

    tokens = word_tokenize(text.lower())

    vector = []
    for token in tokens:
        if vocab_space.get(token):
            vector.append(vocab_space[token])
        else:
            vector.append(vocab_space['<UNK>'])

    return vector


text_to_vector("You have to attempt all the course assessments.", vocab_space)

[72, 69, 288, 186, 66, 194, 152, 27, 233]

In [8]:
sentences = document.split('\n')

vector_sentences = []
for sentence in sentences:
    vector_sentence = text_to_vector(sentence, vocab_space)
    vector_sentences.append(vector_sentence)

len(vector_sentences)

77

training sequences

In [9]:
training_sequences = []
for id, vector_sentence in enumerate(vector_sentences):
    for i in range(1, len(vector_sentence)):
        training_sequences.append(vector_sentence[:i + 1])

len(training_sequences)

942